In [2]:
import pandas as pd
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from tqdm import tqdm


In [22]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.iloc[:, 1:-1].values.astype('float32')
        self.y = df.iloc[:, -1].values
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [12]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)

        # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X
        



In [13]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [20]:
# train the model
def train_model(train_dl, model):
    size = len(train_dl.dataset)
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
        # enumerate epochs
    for epoch in tqdm(range(100),desc='Training Epochs'):
        print(f"Epoch {epoch+1}\n-------------------------------")
        # enumerate mini batches
        for batch, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            #if batch % 100 == 0:
            loss, current = loss.item(), batch * len(inputs)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [15]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc


In [16]:
# make a class prediction for one row of data
def predict(row, model):
# convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat


In [23]:
# prepare the data
path = 'Iris.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(4)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction
row = [5.1,3.5,1.4,0.2]
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

100 50


Training Epochs:  14%|█▍        | 14/100 [00:00<00:00, 138.61it/s]

Epoch 1
-------------------------------
loss: 1.225657 [    0/  100]
loss: 1.036347 [   32/  100]
loss: 1.118782 [   64/  100]
loss: 0.825049 [   12/  100]
Epoch 2
-------------------------------
loss: 0.976485 [    0/  100]
loss: 0.956556 [   32/  100]
loss: 0.955060 [   64/  100]
loss: 1.218741 [   12/  100]
Epoch 3
-------------------------------
loss: 0.933215 [    0/  100]
loss: 0.951111 [   32/  100]
loss: 0.984698 [   64/  100]
loss: 0.656158 [   12/  100]
Epoch 4
-------------------------------
loss: 1.020600 [    0/  100]
loss: 0.925341 [   32/  100]
loss: 0.865654 [   64/  100]
loss: 0.826126 [   12/  100]
Epoch 5
-------------------------------
loss: 0.984448 [    0/  100]
loss: 0.819578 [   32/  100]
loss: 0.922507 [   64/  100]
loss: 0.811416 [   12/  100]
Epoch 6
-------------------------------
loss: 1.031016 [    0/  100]
loss: 0.782442 [   32/  100]
loss: 0.885972 [   64/  100]
loss: 1.027918 [   12/  100]
Epoch 7
-------------------------------
loss: 0.912448 [    0/  

Training Epochs:  47%|████▋     | 47/100 [00:00<00:00, 142.88it/s]

Epoch 32
-------------------------------
loss: 0.907809 [    0/  100]
loss: 0.890174 [   32/  100]
loss: 0.876183 [   64/  100]
loss: 0.800628 [   12/  100]
Epoch 33
-------------------------------
loss: 0.847796 [    0/  100]
loss: 0.854638 [   32/  100]
loss: 0.941943 [   64/  100]
loss: 1.035759 [   12/  100]
Epoch 34
-------------------------------
loss: 0.827518 [    0/  100]
loss: 0.877930 [   32/  100]
loss: 0.967545 [   64/  100]
loss: 0.801450 [   12/  100]
Epoch 35
-------------------------------
loss: 0.857170 [    0/  100]
loss: 0.935501 [   32/  100]
loss: 0.881703 [   64/  100]
loss: 0.788688 [   12/  100]
Epoch 36
-------------------------------
loss: 0.906653 [    0/  100]
loss: 0.859407 [   32/  100]
loss: 0.879537 [   64/  100]
loss: 1.031815 [   12/  100]
Epoch 37
-------------------------------
loss: 0.776348 [    0/  100]
loss: 0.849162 [   32/  100]
loss: 1.025154 [   64/  100]
loss: 1.015498 [   12/  100]
Epoch 38
-------------------------------
loss: 0.879850 [ 

Training Epochs:  77%|███████▋  | 77/100 [00:00<00:00, 141.82it/s]

loss: 0.924268 [   32/  100]
loss: 0.814249 [   64/  100]
loss: 0.785550 [   12/  100]
Epoch 61
-------------------------------
loss: 0.869279 [    0/  100]
loss: 0.831930 [   32/  100]
loss: 0.786589 [   64/  100]
loss: 0.718674 [   12/  100]
Epoch 62
-------------------------------
loss: 0.784968 [    0/  100]
loss: 0.826048 [   32/  100]
loss: 0.768964 [   64/  100]
loss: 0.836416 [   12/  100]
Epoch 63
-------------------------------
loss: 0.742478 [    0/  100]
loss: 0.760486 [   32/  100]
loss: 0.804878 [   64/  100]
loss: 0.810441 [   12/  100]
Epoch 64
-------------------------------
loss: 0.731596 [    0/  100]
loss: 0.801129 [   32/  100]
loss: 0.740474 [   64/  100]
loss: 0.764075 [   12/  100]
Epoch 65
-------------------------------
loss: 0.761563 [    0/  100]
loss: 0.741051 [   32/  100]
loss: 0.754640 [   64/  100]
loss: 0.867577 [   12/  100]
Epoch 66
-------------------------------
loss: 0.734542 [    0/  100]
loss: 0.694592 [   32/  100]
loss: 0.778426 [   64/  100]


Training Epochs: 100%|██████████| 100/100 [00:00<00:00, 134.41it/s]

loss: 0.700245 [    0/  100]
loss: 0.661331 [   32/  100]
loss: 0.616117 [   64/  100]
loss: 0.718434 [   12/  100]
Epoch 88
-------------------------------
loss: 0.718677 [    0/  100]
loss: 0.650289 [   32/  100]
loss: 0.627144 [   64/  100]
loss: 0.594709 [   12/  100]
Epoch 89
-------------------------------
loss: 0.643723 [    0/  100]
loss: 0.709436 [   32/  100]
loss: 0.659083 [   64/  100]
loss: 0.565838 [   12/  100]
Epoch 90
-------------------------------
loss: 0.676741 [    0/  100]
loss: 0.639964 [   32/  100]
loss: 0.633888 [   64/  100]
loss: 0.571676 [   12/  100]
Epoch 91
-------------------------------
loss: 0.651196 [    0/  100]
loss: 0.655385 [   32/  100]
loss: 0.646694 [   64/  100]
loss: 0.573125 [   12/  100]
Epoch 92
-------------------------------
loss: 0.606110 [    0/  100]
loss: 0.691662 [   32/  100]
loss: 0.633681 [   64/  100]
loss: 0.599855 [   12/  100]
Epoch 93
-------------------------------
loss: 0.670094 [    0/  100]
loss: 0.584032 [   32/  100]
